# このページの目的

Google IMEで同時打鍵配列が利用できるのをご存知でしょうか。同時打鍵配列はローマ字を使った入力に比べタイピングのキーを押す回数を減らすことができ、かな入力に比べ指が無理なく動くという特徴があります。しかしこの同時打鍵配列を Google IME で利用するには (現状では) Google IME のバイナリ設定ファイルに変更を加える必要があり、それが同時打鍵配列導入の障壁になっていました。

このページは、そのような設定の変更をマウスクリックだけで行うために作成されました。お使いのマシンにある Google IME の設定ファイル (`config1.db`) をアップロードし、最低限の項目を選択することで同時打鍵配列関連の設定が施されたファイルがダウンロードされます。

Google IME においては Google IME で元々提供されているローマ字テーブルを変更することで同時打鍵配列が実現されます。このページでは新下駄配列という同時打鍵配列を実現するために用意されたテーブルを使用、または同時打鍵に対応させるために記述された独自配列のローマ字テーブルを使って同時打鍵を実現することができます。それ以外の設定は元のファイルにあったものが引き継がれることになります。独自の同時打鍵配列をローマ字テーブルに記述したい場合は、このページ下部にある「ローマ字テーブルと同時打鍵配列」を参照ください。

# 注意と免責

* このページは Google により作成されたものではなく、同時打鍵配列という沼にハマってしまった個人が作成したものです。
* このページで行われる一連の作業には Google アカウントが必要です。
* 利用に際しては、それに起因するいかなる損害についても作者に責を負わせないことに同意ください。
* 何らかの問題が発生した時に備えて、元の設定ファイル (`config1.db`) のコピーを手元に残して置くようにしてください。
* このページは親指シフトについて扱っていません。現状の Google IME 単体で親指シフトが実現できるかも不明です (多分無理)。

# 1. `config1.db` のアップロード

最初に手元 (ローカルマシン) にある `config1.db` をアップロードします。`config1.db` は以下の場所に格納されているはずです:

* Windows:
    * `C:\Users\USERNAME\AppData\LocalLow\Google\Google Japanese Input\`
* Linux (Google IME ではなく `mozc` を利用): 
    * `$HOME/.mozc/` または `$HOME/.config/mozc/`
* macOS: 
    * `$HOME/Library/Application\ Support/Google/JapaneseInput/` (Finder アプリにて `alt+option` キーを押下しながら `移動` メニューを選択することで `$HOME/Library/` が表示されます)

以下のセルにある再生ボタンをクリックすると、セルの下に "Browse" (または "参照") というボタンが出現するはずなので、そのボタンをクリックして、上記の場所にある `config1.db` を選択してください。

In [ ]:
#@title ファイルのアップロード
from google.colab import files
from google.colab import runtime
import re
import sys

uploaded = files.upload()
if(len(uploaded.keys()) != 1):
    print('Please upload only 1 file')
    print('アップロードするファイルは 1 つに限定してください')
    runtime.unassign()


# 2. タイムアウトの指定と配列の選択、変更後のファイルダウンロード

次は同時打鍵を判定するためのタイムアウトを指定します。下にある `timeout_int` の値を入力してください。この値は整数しか指定できないようになっており、下限値を `50`、上限値を `2000` としています。デフォルトの値は `80` でこれは「あるキーを押してから次のキーを押すまでの時間が 80 ミリ秒以下であれば同時打鍵と見なす」ことを意味しています。

新下駄配列をお使いになりたい場合、`roman_table` の項目から新下駄配列を選択してください。`逐次打鍵` が含まれる項目を選択した場合、同時打鍵の設定は行われず (原理としてはローマ字入力と同じような) 逐次打鍵の配列が設定されます。既に独自の同時打鍵配列のテーブルを用意されている場合、`独自配列のローマ字テーブルを使用` を選択してください。

一連の設定が完了したところで再生ボタンをクリックすることで、変更が施されたファイルのダウンロードが始まります。ダウンロードが開始されるまでには最大数分かかることがあります。

In [ ]:
!apt-get -y install protobuf-compiler python-pil python-lxml  > /dev/null 2>&1
print('Protocol Buffer パッケージのセットアップ完了')
!git clone https://github.com/google/mozc.git > /dev/null 2>&1
print('Google Mozc repo のダウンロード完了')

!git clone https://github.com/kirameister/keyboard_layouts.git > /dev/null 2>&1
additional_roman_entry = '{!}\\\\t\\\\t\\\\tNoTransliteration\\\\n'
# print("必要な環境が整いました")

min_timeout = 50
max_timeout = 2000
timeout = 80 #@param {type:"integer"}
if(timeout < min_timeout):
    timeout = min_timeout
if(timeout > max_timeout):
    timeout = max_timeout

新下駄配列_JIS = 'keyboard_layouts/simultaneous/shingeta_JIS_roman_table.txt'
新下駄配列_US = 'keyboard_layouts/simultaneous/shingeta_US_roman_table.txt'
AZIK_逐次打鍵 = 'keyboard_layouts/sequential/AZIK.txt'
独自同時打鍵配列 = 'empty_simultaneous'
roman_table = 新下駄配列_JIS #@param ["新下駄配列_JIS", "新下駄配列_US", "AZIK_逐次打鍵", "独自同時打鍵配列" ] {type:"raw"}
# following 4 lines are most probably not necessary, but just in case..
simul_flag = False
if(re.search('simultaneous', roman_table)):
    simul_flag = True
    roman_table = None

with open ('./mozc/timeout_line.txt', 'w') as f:
    print(f"composing_timeout_threshold_msec: {timeout}", file=f)

for file in uploaded.keys():
    roman_table_stream = !cat $file | protoc --proto_path=mozc/src/protocol --decode "mozc.config.Config" mozc/src/protocol/config.proto | grep "custom_roman_table"
    if(roman_table):
        new_roman_table_stream = !cat $roman_table
    else:
        new_roman_table_stream = !cat $file | protoc --proto_path=mozc/src/protocol --decode "mozc.config.Config" mozc/src/protocol/config.proto | grep "custom_roman_table"
    if(len(new_roman_table_stream) < 1): # custom_roman_table did not exist..
        print("カスタムローマ字テーブルを検出できませんでした。終了します")
    else:
        print(f"{file} を更新中..")
        new_roman_table = new_roman_table_stream[0]
        if(not re.search(additional_roman_entry, new_roman_table) and simul_flag):
            new_roman_table = re.sub('^custom_roman_table: "(.*)"', fr'custom_roman_table: "{additional_roman_entry}\1"', new_roman_table)
        with open('./mozc/new_roman_table.txt', 'w') as f:
            print(new_roman_table, file=f)
        !cat $file | protoc --proto_path=mozc/src/protocol --decode "mozc.config.Config" mozc/src/protocol/config.proto | grep -v "custom_roman_table" | grep -v "composing_timeout_threshold_msec" > ./mozc/config1.txt
        if(simul_flag):
            !cat ./mozc/config1.txt ./mozc/timeout_line.txt ./mozc/new_roman_table.txt | protoc --proto_path=mozc/src/protocol --encode "mozc.config.Config" mozc/src/protocol/config.proto > ./mozc/config1.db
        else:
            !cat ./mozc/config1.txt ./mozc/new_roman_table.txt | protoc --proto_path=mozc/src/protocol --encode "mozc.config.Config" mozc/src/protocol/config.proto > ./mozc/config1.db

        print('更新が完了しました。ファイルのダウンロードが始まります')
        files.download('./mozc/config1.db')
#runtime.unassign()

# 3. ダウンロードされたファイルの移動と再起動

これで変更後の `config1.db` がダウンロードされました (この時点では、お使いの環境に変更はなされていません)。念の為アップロードした `config1.db` のバックアップを取り、ダウンロードされたファイルを (アップロード時に指定した) 元のファイルの場所に置き直し、お使いのマシンを再起動してください。これで Google IME にて選択された配列が使えるようになるはずです。

# (おまけ) Google IME のローマ字テーブルと同時打鍵配列

Google IME ではローマ字テーブルの機能を利用することで同時打鍵入力を実現できます。

同時打鍵の話をする前に、広く世に使われている (Google IME にデフォルトで用意されている) ヘボン式のローマ字テーブルについて考えてみましょう。デフォルト状態の Google IME の `[プロパティー]` を開き、`[全般]` タブにある `ローマ字テーブル` の `[カスタマイズ]` ボタンをクリックすると以下の様なエントリー (抜粋) が表示されるはずです。

|入力|出力|次の入力|
|:-|:-|:-|
|tt|っ|t|
|ta|た||

上のテーブルの内容からは以下のような振る舞いが期待できます:

1. 何も入力がなされていない状態から `t` を入力すると、入力の「メモリー」に `t` が入る。
2. 次にもう一度 `t` を入力すると、入力のメモリーには `tt` が入る。これは上のテーブルの最初のエントリーにマッチするので、画面には `っ` が出力され、入力のメモリーには `t` が入った状態になる。
3. 次に `a` が入力されるとメモリーの内容は `ta` となり、これはテーブル 2 行目のエントリーに一致するので `た` が出力され、入力のメモリーは空になる。

このようにして Google IME では「複数のキーストロークからひらがなを出力」するように実装されています。

翻って同時打鍵配列ですが、例えば新下駄配列では `k` だけを押すと「い」が、`kd` を高速に（同時に）押すと「れ」が入力されます。これら 2 つの状態を区別するために、`k` を押してしばらく待つ、と、`k と `d` を高速に押した状態をテーブルに書かないといけません。そのため、Google IMEでは {!} と書くことで「しばらく入力がなかった」ということを記述できます。この「しばらく」が設定項目の「タイムアウト」の時間となります。

上のような振る舞いは Google IME では以下のようローマ字テーブルを記述することで実現できます (以下は新下駄配列からの抜粋):

|入力|出力|次の入力|
|:-|:-|:-|
|k||い|
|い{!}|い||
|d||か|
|か{!}|か||
|かk|れ||
|いd|れ||

ここで注目すべきは `{!}` という文字列で、これは一定時間次の入力がなかった場合に出力として確定することを意味します (漢字変換の出力ではないことに注意)。このページで行われる同時打鍵の設定を有効にした場合、上のテーブルからは以下のような挙動が期待できます。

* `k` がキーボードから入力されると、一旦入力のメモリーに `い` を入れる (1 行目)。
* `い` がメモリーに入っている状態で一定時間次の入力がないと、`い` を出力として確定し、メモリーを空にする (2 行目)。
* `い` がメモリーに入っている状態で `d` が入力されると、(すでにある `い` を消して) `れ` を出力し、メモリーを空にする (6 行目)。

つまりこのような内容で Google IME のローマ字テーブルを変更し、このページにて設定ファイルを更新すると、独自の同時打鍵配列を Google IME に閉じた形で実現することができるというわけです :)